## This code implements the REINFORCE algorithm
![ ](images/reinforce.png)


The REINFORCE Algorithm is simplestt Policy Gradient algorithms in deep renforcement learning <br>
Policy gradient algorithms directly diffierintate the RL objective  
$$ J(\theta) = \mathop{\mathbb{E}}_{\pi_{\theta}}{[\sum_{t=1}^T}r(s_t,a_t)] $$
from the definitation of Expectation 
$$J(\theta) = \int \pi_\theta(a_t|s_t) \sum_{t=1}^Tr(s_t,a_t)  {d}(P(a_t|s_t) $$
This objective can be defferinated to be 
$$\nabla_\theta J(\theta) =\int \nabla_\theta \pi_\theta(a_t|s_t) \sum_{t=1}^Tr(s_t,a_t) {d}(P(a_t|s_t) $$
We know from calculus that 
$${d}\log{n} = \frac{{d}n}{n}$$
Hence the policy gradient is
$$\nabla_\theta J(\theta) =\int  \pi_\theta(a_t|s_t) \nabla_\theta \log {\pi_\theta(a_t|s_t)} \sum_{t=1}^Tr(s_t,a_t) {d}(P(a_t|s_t) $$
Which can be written as Expectation
$$\nabla_\theta J(\theta)  = \mathop{\mathbb{E}}_{\pi_{\theta}}[\nabla_\theta \log {\pi_\theta(a_t|s_t)}\sum_{t=1}^Tr(s_t,a_t)] $$

From a practicial point of view the general structre of any Reinfrocement learning algorithm is shown below
![image.png](images/Anatomy.png)
for this vareint of REINFORCE algorithms the implementation structure will be based on the framework shown above  </br >  
1. Generate Data by running the most recent policy, specifically this step should return states, rewards, and actions for each batch of training episode
2. Return estimation will implement Monte Carlo policy evaluation with discount factos causality 
$$ \hat{Q}(s_t,a_t) = \sum_{{t'}=t}^T r(s_{t'},a_{t'})$$
3. In this step the gradient ascent will be performed on the policy after sampling the gradient and taking the <br>
of the sample <br>
$$\theta = \theta + \alpha\nabla{J(\theta}) $$ where J is the RL objective

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as opt
import torch.nn.functional as F
import numpy as np

from torch.utils.tensorboard import SummaryWriter

In [ ]:
# cartpole problem with discrete actions
# Network archetacture

class SampleGeneration():
    @staticmethod
    @torch.no_grad()
    def generate_samples(network, env='CartPole-v1', N=4):
        states_N = []
        actions_N = []
        rewards_N = []
        env = gym.make(env)
        for trajectory in range(N):
            state = env.reset()
            states_N.append(state)
            rewards = []
            done = False
            while not done:
                state_t = torch.tensor(state.astype(np.float32)).unsqueeze(0)
                action_logits = network(state_t)
                actions_prob = network.softmax(action_logits)
                action = torch.multinomial(actions_prob, 1).item()
                actions_N.append(action)
                state, reward, done, _ = env.step(action)
                rewards.append(reward)
                if not done:
                    states_N.append(state)
            rewards_N.append(np.array(rewards))
            state_stack = np.stack(states_N)
        return (state_stack, rewards_N, np.array(actions_N))

In [ ]:
# create Network
class Policy_Network(nn.Module):
    def __init__(self):
        super(Policy_Network, self).__init__()
        
        self.fc1 = nn.Linear(in_features=4, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=2)
        
    def forward(self, x):
        # layer 1
        x = F.relu(self.fc1(x))
        # output layer
        x = self.out(x)
        return x
    def softmax(self, logits):
        return F.softmax(logits, dim=1)

In [ ]:
def list_to_torch_tensor(List):
    l = []
    for element in List:
        for r in element:
            l.append(r)
    return torch.tensor(l, dtype=torch.float32)

In [ ]:
class ReturnEstimator():
    # reward_to_go
    @staticmethod
    def estimate_return(rewards):
        gamma = 0.99
        res = [[] for i in range(len(rewards))]
        for i in range(len(rewards)):
            sum_r = 0.0
            for r in rewards[i]:
                sum_r *= gamma
                sum_r += r
                res[i].append(sum_r)
            res[i].reverse()
        return np.array(res)

In [ ]:
def improve_policy(network, states, rewards_to_go, actions, optimizer, tb, step):
    
    optimizer.zero_grad()
    states_t = torch.tensor(states, dtype=torch.float32)
    q = list_to_torch_tensor(rewards_to_go)
    q_t = torch.tensor(q, dtype=torch.float32)
    
    logits = network(states_t)
    actions_log_probs = F.log_softmax(logits, dim=1)
    selected_actions_log_probs_t = actions_log_probs[range(len(actions_log_probs)), actions]
    loss = selected_actions_log_probs_t * q_t
    loss = -loss.mean()
    tb.add_scalar('loss', loss, step)
    loss.backward()
    optimizer.step()

In [ ]:
@torch.no_grad()
def test_policy(network, env="CartPole-v1"):
    runs = 5
    total_reward = 0.0
    env = gym.make(env)
    for run in range(runs):
        state = env.reset()
        done = False
        while not done:
            state_t = torch.tensor(state, dtype=torch.float32)
            action_logits = network(state_t)
            action = action_logits.argmax(dim=0).item()
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / runs

In [ ]:
# policy imporvment
# hyperparameters
episodes_num = 10000
# batch size
N = 4
#######
env = 'CartPole-v1'
## track the test rewards of the last 100 runs
rewards_100 = []
lrs = 0.001
for lr in lrs:
    policy = Policy_Network()
    tb = SummaryWriter(comment=f"-lr={lr}")
    states = torch.tensor(gym.make(env).reset(), dtype=torch.float32)
    # add the policy graph to tensorboard
    tb.add_graph(policy, states)
    print(policy)
    # Adam optimizer was used you can change it with your favorite optimizer
    optimizer = opt.Adam(policy.parameters(), lr=lr)
    for i in range(episodes_num):
        # run the policy and collect data
        states, rewards, acttions = SampleGeneration.generate_samples(policy, env=env,N=N)
        # estimate the return (Monte carlo evaluation)
        q = ReturnEstimator.estimate_return(rewards)
        # imporove the polciy by taking a step in the gradient ascent direction
        improve_policy(policy, states, q, actions, optimizer, tb, i )
        # test the policy by rinning the env 5 times and average the result
        test_reward = test_policy(policy, env=env)
        rewards_100.append(test_reward)
        # write the reward to tensorboard
        tb.add_scalar('reward', test_reward, i)

        if len(rewards_100) >= 100:
            reward_100 = sum(rewards_100) / 100.0
            tb.add_scalar('reward_100', reward_100, i)
            rewards_100 = []
            # winning test
            if reward_100 >= 450:
                print("done")
                break
        for name, param in policy.named_parameters():
            tb.add_histogram(f'{name}', param, i)
            tb.add_histogram(f'{name}.grad', param.grad, i)

## Agent Performence
<video width=320 height=240 controls src="recording/test.mp4" />

## Tips and comment
1. The performance of the algorithm can be improved by using any baseline which is not a function of actions
2. To reduce the variance of the policy gradient TD learning can be used to evaluate (But it is baised)
3. due to my lack of computing power and inability to acess the cloud from my country I test the code on simple
 environemnt but you can modify it to work with GPUs it is easy.
4. always check the gradient values to ensure the learning process is healthy

## References
1. CS 285 at UC Berkeley
Deep Reinforcement Learning
http://rail.eecs.berkeley.edu/deeprlcourse/static/slides/lec-5.pdf
2. Reinforcement Learning: An Introduction by Richard S. Sutton
and Andrew G. Barto, chapter 13 http://incompleteideas.net/book/the-book-2nd.html